In [195]:
import numpy as np 
import pandas as pd 


In [196]:
df = pd.read_csv("hacktrain.csv")
df
df=df.drop('ID',axis=1)

In [197]:

df2=df.copy()


In [198]:
df2.isnull().sum()

Unnamed: 0       0
class            0
20150720_N     560
20150602_N    1200
20150517_N     800
20150501_N     960
20150415_N     480
20150330_N    1120
20150314_N     720
20150226_N    1360
20150210_N     640
20150125_N    1040
20150109_N     880
20141117_N    1280
20141101_N     400
20141016_N    1440
20140930_N     800
20140813_N     560
20140626_N    1600
20140610_N     480
20140525_N     720
20140509_N     880
20140423_N    1760
20140407_N     640
20140322_N    1120
20140218_N    1440
20140202_N     560
20140117_N    1200
20140101_N     400
dtype: int64

In [199]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge 


label_encoder = LabelEncoder()
df2['class'] = label_encoder.fit_transform(df2['class'])

X = df2.drop(columns=['class'])
y = df2['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

iterative_imputer = IterativeImputer(
    estimator=BayesianRidge(), 
    max_iter=10,
    random_state=42
)

X_train_imputed = iterative_imputer.fit_transform(X_train)
X_test_imputed = iterative_imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

smote_target = SMOTE(
        sampling_strategy={4:100},  
        random_state=42
    )
X_train_smote, y_train_smote = smote_target.fit_resample(X_train_scaled, y_train)


model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=100,
)
model.fit(X_train_smote, y_train_smote)

from sklearn.metrics import precision_recall_fscore_support

orchard_threshold = 0.7  
probs = model.predict_proba(X_test_scaled)

y_pred_custom = []
for row in probs:
    if row[4] >= orchard_threshold:
        y_pred_custom.append(4)
    else:
        row[4] = -1  
        y_pred_custom.append(np.argmax(row))


y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred_custom, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

        farm       0.87      0.88      0.88       252
      forest       0.99      0.99      0.99      1848
       grass       0.86      0.73      0.79        59
  impervious       0.85      0.89      0.87       201
     orchard       1.00      0.33      0.50         9
       water       0.86      0.81      0.83        31

    accuracy                           0.96      2400
   macro avg       0.91      0.77      0.81      2400
weighted avg       0.96      0.96      0.96      2400



C:\Users\Ananya\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [172]:
test_data = pd.read_csv("hacktest.csv")
ID = test_data['ID']
features = test_data.drop(columns=['ID'])
features_scaled = scaler.transform(features)

C:\Users\Ananya\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [173]:
ID=test_data['ID']
test_data.drop(['ID'],axis=1,inplace=True)

In [174]:
y_test = model.predict(features_scaled)
y_test
y_decoded = label_encoder.inverse_transform(y_test)
y_decoded

array(['forest', 'forest', 'forest', ..., 'water', 'water', 'water'],
      dtype=object)

In [175]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})
result['class']

0       forest
1       forest
2       forest
3       forest
4       forest
         ...  
2840     water
2841     water
2842     water
2843     water
2844     water
Name: class, Length: 2845, dtype: object

In [176]:
result.to_csv("submission13.csv", index=False)

In [177]:
df2=pd.read_csv("submission13.csv")

In [178]:
df2.head(30)

,ID,class
0,1,forest
1,2,forest
2,3,forest
3,4,forest
4,5,forest
5,6,forest
6,7,forest
7,8,forest
8,9,forest
9,10,forest
